In [5]:
spark.streams.active

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
userSchema = StructType() \
        .add("window",StructType()\
             .add("start",TimestampType())\
             .add("end",TimestampType()))\
        .add("subject", StringType())\
        .add("count", LongType())

In [7]:
raw_data = spark\
.readStream.format("parquet")\
.schema(userSchema)\
.load("/cms/users/carizapo/ming/groupdata");
raw_data.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- subject: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
raw_data_flow.lastProgress()

In [4]:
raw_data_flow = raw_data.writeStream.queryName("hdfs").outputMode("Append").format("memory").start()

In [ ]:
raw_data_flow.stop()

In [ ]:
alerts = spark.sql("select * from hdfs")
alerts.show()

In [6]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [2]:
# Build the model (cluster the data)
# clusters = KMeans.train(raw_data, 2, maxIterations=10, initializationMode="random")
